In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler 
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
df=pd.read_csv('/home/mandana/Data/slsa-surfguard-NEW_MG3_final1_wave_wind_current_rename_cgp.csv')
df['time']=pd.to_datetime(df['time'])
df.head(2)

,time,lat,lon,species,beach_key_lower,stings,attendance,duration,species.1,state,...,cos_dir_wave,sfcWind_wind,ts_wind,uas_wind,vas_wind,salt_crt,temp,u_crt,v_crt,stings_binary
0,2009-01-01,-26.4065,153.1122,Physalia (Bluebottle),qld1539a,3.0,20,5.5,Physalia (Bluebottle),QLD,...,0.573162,5.585938,299.656250,-0.483398,-5.183594,35.330060,26.319626,0.040895,-0.134892,1
1,2009-01-01,-26.6096,153.0984,Physalia (Bluebottle),qld1543d,16.0,789,6.0,Physalia (Bluebottle),QLD,...,0.622495,5.964844,299.640625,-0.352539,-5.555664,35.331738,26.234009,0.077517,-0.169683,1


In [ ]:
dff=pd.read_csv('/home/mandana/Data/slsa-surfguard-NEW_MG3_final_Bin.csv')
dff['time'] = pd.to_datetime(dff['date'])

In [ ]:
df3=pd.read_csv('/home/mandana/Data/slsa-surfguard-NEW_MG3_final1_wave_wind_current_rename_cgp_merged_wind.csv')
df3['time'] = pd.to_datetime(df3['time'])
# df3['beach_key_lower']= df3['beach_key_'].str.lower()
df3=pd.merge(df3, dff[['embaymentisation', 'orientation','time', 'beach_key_lower']], on=['beach_key_lower','time'], how='left',)

df3['stings_binary'] = 0
df3.loc[df3['stings'] != 0, 'stings_binary'] = 1


df3['day_of_year'] = df3['time'].dt.dayofyear 
df3['is_weekend'] = df3['time'].dt.weekday >= 5  
df3 = df3[df3['time'].dt.month.isin([1,2,3,4,8,9,10,11])].copy() # Filter for all months

df3 = df3.sort_values(['beach_key_lower', 'time'])
df3=df3[df3['state'] == 'QLD'].copy()
df3=df3[df3['stings']!=1].copy()  # Exclude rows where stings is 1

# df3 = df3[df3['species'] == 'Physalia (Bluebottle)'].copy()



In [ ]:
# df3=df3[(df3['lat'] >= -28.18) & (df3['lat'] <= -27.90) &
#          (df3['lon'] >= 153.40) & (df3['lon'] <= 153.58)].copy()

In [ ]:
df3=df3[(df3['lat'] >= -26.88) & (df3['lat'] <= -26.3) &
         (df3['lon'] >= 153) & (df3['lon'] <= 153.2)].copy()



df3=df3[~df3['beach_key_lower'].isin(['qld1532','qld1543a','qld1544b'])].copy()

In [ ]:
# Reload the processed dataset
file_path = '/home/mandana/Data/slsa-surfguard-NEW_MG3_final1_wave_wind_current_rename_cgp_merged_wind.csv'


df = pd.read_csv(file_path)
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values(['beach_key_lower', 'time'])


# df['stings_binary2'] = np.where(df['stings'] > 3, 1, 0)


target = "stings_binary"
# target = "stings_binary2"

In [ ]:
# temp=pd.read_csv('/home/mandana/Data/BB_edited_locations.csv')

In [ ]:
# beach_list=temp['beach_key_lower']
# beach_list=['qld1532']
# beach_list=['qld1554']

In [ ]:
# beach_list = [
#     "nsw196", "nsw245a", "nsw250a", "nsw259", "nsw317b", "nsw366", "nsw206",
#     "nsw278", "nsw280", "nsw303", "nsw324", "nsw164", "nsw287b", "nsw297b",
#     "nsw468", "nsw625", "qld0697", "nsw268b", "nsw300a", "nsw310b", "nsw304",
#     "nsw317c", "nsw013a", "nsw274", "nsw302", "nsw326", "qld1498", "qld1525",
#     "nsw004", "nsw223", "nsw300b", "nsw307", "nsw308", "nsw323", "nsw327b",
#     "qld1570", "qld1363", "qld1597b", "nsw285b", "qldmt05", "qld1433", "qld1509",
#     "qld1597a", "qld1377", "qld1591h", "qld1543a", "qld1591i", "qld1593a", "nsw275",
#     "qld1593b", "qld1591g", "qld1595", "qld1597c", "qld1591e", "qld1593c", "qld1599a",
#     "qld1549f", "qld1592a", "qld1588a", "qld1591c", "qld1591d", "qld1549c", "qld1598",
#     "qld1591f", "qld1531", "qld1592b", "qld1503", "qld1501", "qld1599b", "nsw265",
#     "qld1555", "qld1600", "qld1591b", "qld1550", "qld1544b", "qld1471", "qld1549b",
#     "qld1539b", "qld1551", "qld1554", "qld1581", "qld1545a", "qld1545b", "qld1543c",
#     "qld1547", "qld1543d", "qld1539d", "qld1583", "qld1532", "qld1539e", "qld1529",
#     "qld1556", "qld1539a"]


In [ ]:
df=df[(df['stings']>1)| (df['stings']==0)].copy() # Filter for stings > 5, increased from 22 to 42
df = df[df['species'] == 'Physalia (Bluebottle)'].copy() #increased from 22 to 42
# df=df[df['state'] == 'QLD'].copy()  # Filter for QLD, only qld42 and if i include nsw too will be 47



df = df[df['time'].dt.month.isin([1])].copy() #Increase from 47 to 60, Jan alone 68%, dec52%


# df=df[df['beach_key_lower'].isin(beach_list)].copy()
df['day_of_year'] = df['time'].dt.dayofyear # 3% more, becasue i included only jan
df['is_weekend'] = df['time'].dt.weekday >= 5   #no diff

# features = [ 'ts_wind', 'uas_wind','vas_wind', 'uas','vas','ts',
#              'temp',
#             'is_weekend','lat','lon','embaymentisation', 'orientation', 'day_of_year',]

features = [  'uas','vas','ts',
             'temp',
            'is_weekend','lat','lon','embaymentisation', 'orientation', 'day_of_year',]


df=pd.merge(df, dff[['embaymentisation', 'orientation','time', 'beach_key_lower']], on=['beach_key_lower','time'], how='left',)

In [ ]:

# df3 = df3[df3['time'].dt.month.isin([1])].copy() #Increase from 47 to 60, Jan alone 68%, dec52%


features = [ 'ts_wind', 'uas_wind','vas_wind',
             'temp',
            'is_weekend','lat','lon','embaymentisation', 'orientation', 'day_of_year',
            
            ]  #removing sfcwing increased by 10%

In [ ]:
target = "stings_binary"

In [ ]:
df3.dropna(subset=features + [target], inplace=True)
df3.reset_index(drop=True, inplace=True)

In [ ]:
# scaler = MinMaxScaler()
# df3[features] = scaler.fit_transform(df3[features])

In [ ]:
df3['stings_binary'].value_counts()

### 1-Undersample

In [ ]:
df3['year'] = df3['time'].dt.year

In [ ]:
# Define your training and testing years
train_years = [ 2010, 2011,2012,2012,2013,2014,2015,2016,2017,2018,2019]
test_years = [2020,2021,2022]

# Split the data
train_df = df3[df3['year'].isin(train_years)]
test_df = df3[df3['year'].isin(test_years)]

# Create X and y sets
X_train = train_df[features]
y_train = train_df[target]

X_test = test_df[features]
y_test = test_df[target]


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(df3[features], df3[target], test_size=0.2, stratify=df3[target], random_state=50)



#OverSample
ros = RandomOverSampler(random_state=42)
X_train_res, y_train_res = ros.fit_resample(X_train, y_train)



#Model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train_res, y_train_res)

# Eval
# y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.60).astype(int)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# y_test=df2_filtered[target]

# y_prob = model.predict_proba(df2_filtered[features])[:, 1]
# y_pred = (y_prob >= 0.60).astype(int)
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# y_test=df3['stings_binary2']

# y_prob = model.predict_proba(df3[features])[:, 1]
# y_pred = (y_prob >= 0.60).astype(int)
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)

# Plot it
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# import shap


# explainer = shap.Explainer(model, X_train)
# shap_values = explainer(X_train)


# shap.summary_plot(shap_values, X_train, plot_type="bar")
# shap.summary_plot(shap_values, X_train)

In [ ]:
df_subset=df3.loc[X_test.index,:]

In [ ]:
df_subset.reset_index(drop=True, inplace=True)

In [ ]:
df_subset['y_test'] = y_test.reset_index(drop=True)

In [ ]:
df_subset['y_pred'] = y_pred 

In [ ]:
df_subset['y_pred'] = df_subset['y_pred'].astype(int)
df_subset['y_test'] = df_subset['y_test'].astype(int)


In [ ]:
df_subset

In [ ]:
df_subset.to_csv('/home/mandana/Data/Pred3.csv', index=False)

In [ ]:
df_subset[(df_subset['y_pred'] == 0) & (df_subset['y_test'] == 1) ]['is_weekend'].value_counts()

In [ ]:
df_subset[(df_subset['y_pred'] == 0) & (df_subset['y_test'] == 1) ]['beach_key_lower'].value_counts()

In [ ]:
df_subset[(df_subset['y_pred'] == 0) & (df_subset['y_test'] == 1) ]['stings'].value_counts()

In [ ]:
df_merged2= pd.merge(df_subset, df3[[ 'beach_key_lower', 'lat','lon']], on=['beach_key_lower'], how='left')

In [ ]:
df_merged2.to_csv('/home/mandana/Data/Jan_stings_test1_pred_merged.csv', index=False)